In [1]:
#omgamganapathayenamaha
#omgamganapathayenamaha
#omgamganapathayenamaha
#omgamganapathayenamaha
#omgamganapathayenamaha

In [2]:
import sagemaker

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [3]:
from sagemaker.serializers import CSVSerializer
predictor = sagemaker.predictor.Predictor(endpoint_name='xgboost-aws-v1',
                                          serializer=CSVSerializer()
                                         )

In [4]:
import pandas as pd
import numpy as np

In [6]:
import joblib
class sparse_to_dense:
    def fit(self, X, y=None, **fit_params):
        return self
    def transform(self, X, y=None, **fit_params):
        return X.toarray()
data_pipeline = joblib.load('data_pipeline_downloaded')

In [7]:
df = pd.read_csv('Restaurant_Reviews.tsv', sep='\t')
df.head()

,Review,Liked
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [8]:
from sklearn.model_selection import train_test_split
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
X_test.shape

(200, 1)

In [12]:
X_test = data_pipeline.transform(X_test)
X_test.shape

(200, 1587)

In [16]:
output = predictor.predict(np.array_split(X_test, 10)[0])

In [20]:
[out for out in output.decode('utf-8').split("\n") if out != ""]

['0.3724002242088318',
 '0.7801342010498047',
 '0.5463298559188843',
 '0.9655318260192871',
 '0.3724002242088318',
 '0.9634009003639221',
 '0.5932883024215698',
 '0.3724002242088318',
 '0.19609250128269196',
 '0.3724002242088318',
 '0.3724002242088318',
 '0.173043891787529',
 '0.3724002242088318',
 '0.2644158899784088',
 '0.3724002242088318',
 '0.9531811475753784',
 '0.3724002242088318',
 '0.3724002242088318',
 '0.5614397525787354',
 '0.3724002242088318']

In [21]:
y_pred = []
for chunk in np.array_split(X_test, 10):
    output = predictor.predict(chunk)
    y_pred.extend([out for out in output.decode('utf-8').split("\n") if out != ""])
y_pred

['0.3724002242088318',
 '0.7801342010498047',
 '0.5463298559188843',
 '0.9655318260192871',
 '0.3724002242088318',
 '0.9634009003639221',
 '0.5932883024215698',
 '0.3724002242088318',
 '0.19609250128269196',
 '0.3724002242088318',
 '0.3724002242088318',
 '0.173043891787529',
 '0.3724002242088318',
 '0.2644158899784088',
 '0.3724002242088318',
 '0.9531811475753784',
 '0.3724002242088318',
 '0.3724002242088318',
 '0.5614397525787354',
 '0.3724002242088318',
 '0.5431494116783142',
 '0.8471297025680542',
 '0.3724002242088318',
 '0.3724002242088318',
 '0.3724002242088318',
 '0.3724002242088318',
 '0.6148010492324829',
 '0.3724002242088318',
 '0.4567383825778961',
 '0.3724002242088318',
 '0.5188213586807251',
 '0.965930700302124',
 '0.3724002242088318',
 '0.3724002242088318',
 '0.3724002242088318',
 '0.7801342010498047',
 '0.7690701484680176',
 '0.17461703717708588',
 '0.3724002242088318',
 '0.3724002242088318',
 '0.31363940238952637',
 '0.3724002242088318',
 '0.6918653845787048',
 '0.372400

In [30]:
y_pred = [float(out) for out in y_pred]

In [32]:
y_pred

[0.3724002242088318,
 0.7801342010498047,
 0.5463298559188843,
 0.9655318260192871,
 0.3724002242088318,
 0.9634009003639221,
 0.5932883024215698,
 0.3724002242088318,
 0.19609250128269196,
 0.3724002242088318,
 0.3724002242088318,
 0.173043891787529,
 0.3724002242088318,
 0.2644158899784088,
 0.3724002242088318,
 0.9531811475753784,
 0.3724002242088318,
 0.3724002242088318,
 0.5614397525787354,
 0.3724002242088318,
 0.5431494116783142,
 0.8471297025680542,
 0.3724002242088318,
 0.3724002242088318,
 0.3724002242088318,
 0.3724002242088318,
 0.6148010492324829,
 0.3724002242088318,
 0.4567383825778961,
 0.3724002242088318,
 0.5188213586807251,
 0.965930700302124,
 0.3724002242088318,
 0.3724002242088318,
 0.3724002242088318,
 0.7801342010498047,
 0.7690701484680176,
 0.17461703717708588,
 0.3724002242088318,
 0.3724002242088318,
 0.31363940238952637,
 0.3724002242088318,
 0.6918653845787048,
 0.3724002242088318,
 0.9280167818069458,
 0.247202530503273,
 0.3724002242088318,
 0.1348444521

In [34]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, np.array(y_pred) >= 0.5))

0.71


In [35]:
predictor.delete_endpoint()